In [97]:
import requests
import json
from bs4 import BeautifulSoup
import networkx as nx
import re
from matplotlib import pyplot as plt
import time
import numpy as np

In [98]:
# Load stored course-site HTMLs
with open('valid_courses.json') as f:
    valid_courses = json.load(f)
    
# Load eval links
with open('eval_links.json') as f:
    eval_links = json.load(f)

In [99]:
base_url = "https://kurser.dtu.dk/course/"

cookies_eval = {
    'ASP.NET_SessionId' : "b5klhbigq1mwchpbjawhmqcp",
    '{DTUCoursesPublicLanguage}' : 'en-GB',
}
cookies_grades = {
    'ASP.NET_SessionId' : "ikysu4mskcpfwgkvetw4fs43",
    '{DTUCoursesPublicLanguage}' : 'en-GB'
}

In [100]:
eval_scores = {}
for course_num in eval_links:
    print(course_num)
    
    response_eval   = requests.get(eval_links[course_num]['eval'] , cookies=cookies_eval)
    soup_eval   = BeautifulSoup(response_eval.content,   'html.parser')
    divs = soup_eval.find_all('div', class_='ResultCourseModelWrapper grid_6 clearmarg')
    
    # Questions
    all_weighted_answers = []
    for div in divs[:-1]:
        answers = div.find_all('div', class_='RowWrapper')
        
        # Answers (Helt uenig -> Helt enig)
        answer_i = []
        for answer in answers:
            num_div = answer.find('div', class_='Answer_Result_Background')
            num = num_div.find('span').text
            answer_i.append(int(num))
        
        answer_i = np.array(answer_i)
        scores = np.array([1,2,3,4,5])
        N = sum(answer_i)
        if N == 0:
            continue
        
        weighted_answers = scores@answer_i/N
        all_weighted_answers.append(weighted_answers)

    eval_scores[course_num] = all_weighted_answers

01001
01002
01003
01004
01017
01018
01019
01020
01025
01034
01035
01037
01125
01227
01237
01238
01257
01325
01405
01410
01415
01418
01426
01527
01617
01621
01622
01666
01715
01716
01904
01910
01915
01920
02002
02003
02100
02104
02105
02110
02112
02113
02114
02122
02132
02135
02138
02139
02141
02148
02155
02156
02157
02158
02159
02160
02161
02162
02170
02180
02182
02191
02192
02193
02195
02196
02203
02205
02209
02211
02226
02232
02233
02234
02238
02239
02242
02244
02245
02246
02247
02249
02255
02256
02258
02263
02266
02267
02268
02269
02282
02285
02287
02289
02291
02311
02312
02321
02322
02323
02324
02325
02326
02327
02328
02329
02332
02335
02346
02369
02393
02402
02403
02405
02407
02409
02411
02413
02417
02418
02421
02425
02426
02427
02429
02431
02435
02441
02443
02445
02450
02455
02456
02458
02460
02461
02462
02463
02464
02465
02466
02467
02471
02476
02477
02492
02501
02502
02504
02506
02509
02510
02516
02525
02526
02561
02562
02563
02566
02580
02582
02586
02601
02610
02612
02613
0261

In [101]:
# save dict as json
with open('eval_scores.json', 'w') as file:
    json.dump(eval_scores, file)

In [102]:
grade_scores = {}
for course_num in eval_links:
    print(course_num)
    
    
    response_grades = requests.get(eval_links[course_num]['grade'], cookies=cookies_grades)
    soup_grades = BeautifulSoup(response_grades.content, 'html.parser')
    
    form = soup_grades.find('form', id='karsumForm')
    tds = form.find_all('td')
    td_texts = []
    for td in tds:
        td_text = td.text.replace('\n', ' ').replace('\r', ' ').replace('  ', '').strip()
        td_texts.append(td_text)

    if int(td_texts[3]) <= 3:
        continue

    if td_texts[7] == "Intet eksamensgennemsnit":
        grade_average = None
    else:
        grade_average = float(td_texts[7].split(' ')[0].replace(',', '.'))
    
    grade_scores[course_num] = [
                                int(td_texts[3]),
                                int(td_texts[5].split('(')[0]),
                                grade_average
                                ]

01001
01002
01003
01004
01017
01018
01019
01020
01025
01034
01035
01037
01125
01227
01237
01238
01257
01325
01405
01410
01415
01418
01426
01527
01617
01621
01622
01666
01715
01716
01904
01910
01915
01920
02002
02003
02100
02104
02105
02110
02112
02113
02114
02122
02132
02135
02138
02139
02141
02148
02155
02156
02157
02158
02159
02160
02161
02162
02170
02180
02182
02191
02192
02193
02195
02196
02203
02205
02209
02211
02226
02232
02233
02234
02238
02239
02242
02244
02245
02246
02247
02249
02255
02256
02258
02263
02266
02267
02268
02269
02282
02285
02287
02289
02291
02311
02312
02321
02322
02323
02324
02325
02326
02327
02328
02329
02332
02335
02346
02369
02393
02402
02403
02405
02407
02409
02411
02413
02417
02418
02421
02425
02426
02427
02429
02431
02435
02441
02443
02445
02450
02455
02456
02458
02460
02461
02462
02463
02464
02465
02466
02467
02471
02476
02477
02492
02501
02502
02504
02506
02509
02510
02516
02525
02526
02561
02562
02563
02566
02580
02582
02586
02601
02610
02612
02613
0261

In [104]:
# save dict as json
with open('grade_scores.json', 'w') as file:
    json.dump(grade_scores, file)